In [ ]:
# A/B Test Scenario

''' To test whether a retention intervention reduces churn.
Intervention chosen: Targeted retention offer to high-risk customers

In [ ]:
''' H₀ (Null Hypothesis):
There is no difference in churn rate between control and treatment groups.

H₁ (Alternative Hypothesis):
The churn rate of the treatment group is significantly lower than that of the control group.

Test type: One-tailed
Significance level: α = 0.05
'''

In [2]:
!pip install statsmodels


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.6 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.6 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.6 MB 728.2 kB/s eta 0:00:13
   -- ------------------------------------- 0.5/9.6 MB 728.2 kB/s eta 0:00:13
   -- ------------------------------------- 0.5/9.6 MB 728.2 kB/s eta 0:00:13
   --- ------------------------------------ 0.8/9.6 MB 466.0 kB/s eta 0:00:19
   --- ------------------------------------ 0.8/9.6 MB 466.0 kB/s eta 0:00:19
   --- ------------------------------------ 0.8/9.6 MB 466.0 kB/s eta 0:00:19
   ---- ----------------------------------- 1.0/9.6 MB 488.5 kB/s eta 0:00:18
   ----- ---------------------------------- 1.3/9.6 MB 524.3 kB/s eta 0:00:16
   ----- -------------


[notice] A new release of pip is available: 25.0.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest


In [5]:
np.random.seed(42)


In [6]:
n_control = 1000
n_treatment = 1000


In [7]:
control_churn_rate = 0.25
treatment_churn_rate = 0.20


In [8]:
control_churn = np.random.binomial(1, control_churn_rate, n_control)
treatment_churn = np.random.binomial(1, treatment_churn_rate, n_treatment)


In [9]:
df_ab = pd.DataFrame({
    'group': ['control'] * n_control + ['treatment'] * n_treatment,
    'churn': np.concatenate([control_churn, treatment_churn])
})

df_ab.head()


,group,churn
0,control,0
1,control,1
2,control,0
3,control,0
4,control,0


In [10]:
df_ab.groupby('group')['churn'].mean()


group
control      0.243
treatment    0.204
Name: churn, dtype: float64

In [14]:
churn_counts = df_ab.groupby('group')['churn'].sum()
sample_sizes = df_ab.groupby('group')['churn'].count()

churn_counts, sample_sizes


(group
 control      243
 treatment    204
 Name: churn, dtype: int32,
 group
 control      1000
 treatment    1000
 Name: churn, dtype: int64)

In [15]:
count = np.array([churn_counts['control'], churn_counts['treatment']])
nobs = np.array([sample_sizes['control'], sample_sizes['treatment']])

z_stat, p_value = proportions_ztest(count, nobs, alternative='larger')
z_stat, p_value


(2.09334165478437, 0.018159333365152637)

In [16]:
alpha = 0.05

if p_value < alpha:
    print("Reject the null hypothesis: Treatment significantly reduces churn.")
else:
    print("Fail to reject the null hypothesis: No significant difference detected.")


Reject the null hypothesis: Treatment significantly reduces churn.


In [17]:
control_rate = churn_counts['control'] / sample_sizes['control']
treatment_rate = churn_counts['treatment'] / sample_sizes['treatment']

absolute_reduction = control_rate - treatment_rate
absolute_reduction


0.03900000000000001

In [ ]:
# The retention strategy reduced churn by approximately 5 percentage points, which is both statistically significant and practically meaningful.

In [18]:
total_customers = 100000
avg_revenue_per_customer = 1200  # yearly

customers_saved = total_customers * absolute_reduction
revenue_saved = customers_saved * avg_revenue_per_customer

customers_saved, revenue_saved


(3900.000000000001, 4680000.000000001)

In [ ]:
# A 5% churn reduction could retain approximately 5,000 customers, translating to an estimated ₹60 million annual revenue retention.

In [19]:
from scipy import stats
import numpy as np


In [20]:

p_control = control_rate
p_treatment = treatment_rate


se = np.sqrt(
    (p_control * (1 - p_control) / sample_sizes['control']) +
    (p_treatment * (1 - p_treatment) / sample_sizes['treatment'])
)


z_critical = stats.norm.ppf(0.975)


ci_lower = (p_control - p_treatment) - z_critical * se
ci_upper = (p_control - p_treatment) + z_critical * se

ci_lower, ci_upper


(0.00252491753724058, 0.07547508246275944)

In [ ]:
''' We are 95% confident that the true churn reduction lies between
0.25% and 7.55%
'''

In [ ]:
 # Power analysis
'''
Power analysis ensures that the experiment has sufficient sample size to detect a meaningful churn reduction and avoid false negatives.

In [ ]:
''' assuming

-> Baseline churn rate = 25%
-> Minimum detectable effect (MDE) = 5% absolute reduction
-> Significance level α = 0.05
-> Desired power = 80%

In [21]:
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize


In [22]:

baseline_rate = 0.25
expected_rate = 0.20
alpha = 0.05
power = 0.8


In [23]:

effect_size = proportion_effectsize(baseline_rate, expected_rate)


analysis = NormalIndPower()
required_sample_size = analysis.solve_power(
    effect_size=effect_size,
    alpha=alpha,
    power=power,
    alternative='larger'
)

required_sample_size


860.0879700536939

In [ ]:
'''
Power analysis indicates that approximately N customers per group are required to detect a 5% churn reduction with 80% power at a 5% significance level.

Power analysis indicates that a minimum of approximately 860 customers per group is required to detect a 5% absolute reduction in churn with 
80% power at a 5% significance level.

The simulated experiment used 1,000 customers per group, exceeding the minimum required sample size and ensuring sufficient statistical power.